In [1]:
import numpy as np
import scipy as sp 
import scipy.stats 
import matplotlib.pyplot as plt
import pandas as pd

**Basic Nodes**

**Naive Example (n_feature = 1, m_samples = 10)**

In [2]:
%%writefile basic_nodes.py

import numpy as np 

class add: 
    def __init__(self): 
        self.x, self.y = None, None 
        self.z = None 
    
    def forward(self, x, y):
        self.x, self.y = x, y 
        self.f = self.x + self.y 
        return self.f 
    
    def backward(self, dvoi): 
        dx = dvoi
        dy = dvoi
        return dx, dy 

class subtract: 
    def __init__(self): 
        self.x, self.y = None, None 
        self.z = None 
    
    def forward(self, x, y):
        self.x, self.y = x, y 
        self.f = self.x - self.y 
        return self.f 
    
    def backward(self, dvoi): 
        dx = dvoi 
        dy = -1*dvoi
        return dx, dy

class multiply:
    def __init__(self): 
        self.x, self.y = None, None 
        self.z = None 
    
    def forward(self, x, y):
        self.x, self.y = x, y 
        self.f = self.x * self.y 
        return self.f 
    
    def backward(self, dvoi):
        dx = dvoi*self.y
        dy = dvoi*self.x 
        return dx, dy 

class divide:
    def __init__(self): 
        self.x, self.y = None, None 
        self.z = None 
    
    def forward(self, x, y):
        self.x, self.y = x, y 
        self.f = self.x / self.y 
        return self.f 
    
    def backward(self, dvoi):
        dx = dvoi * 1/self.y
        dy = dvoi * -self.x/(self.y**2) 
        return dx, dy 

class square:
    def __init__(self): 
        self.x = None
        self.z = None 
    
    def forward(self, x):
        self.x = x 
        self.f = self.x**2 
        return self.f 
    
    def backward(self, dvoi):
        dx = dvoi * 2*self.x
        return dx

class mean:
    def __init__(self): 
        self.x = None
        self.z = None 
    
    def forward(self, x):
        self.x = x 
        self.f = np.mean(x) 
        return self.f 
    
    def backward(self, dvoi):
        dx = dvoi * (1/self.x.shape[0])
        return dx

Overwriting basic_nodes.py


In [2]:
import basic_nodes as nodes 

add = nodes.add()
subtract = nodes.subtract() 
multiply = nodes.multiply()
divide = nodes.divide()
square = nodes.square()
mean = nodes.mean()

In [5]:
"""generate_linreg should be used instead"""

X = np.random.randint(low = 0, high = 11, size = 10)
Y = np.random.randint(low = 0, high = 11, size = 10)

weight, bias = -3, -3 

print(X, X.shape[0])
print(Y, Y.shape[0])

[10  1  7  0  2  1  2  7  9  5] 10
[ 5  8  9  1  5  6 10  7  0  9] 10


In [6]:
1/X.shape[0] * np.sum(X), np.mean(X)

(4.4, 4.4)

In [7]:
"""y_hat"""
f1 = multiply.forward(weight, X)
f2 = add.forward(f1, bias)

"""loss"""
f3 = subtract.forward(Y, f2)
f4 = square.forward(f3)

"""cost"""
f5 = mean.forward(f4)

In [8]:
Forward_Propagate = {
    'x': X,
    'y': Y,
    '$\hat{y}$': f2,
    'loss': f4,
    'cost': f5
}

df = pd.DataFrame(data=Forward_Propagate)
df

,x,y,$\hat{y}$,loss,cost
0,10,5,-33,1444,603.6
1,1,8,-6,196,603.6
2,7,9,-24,1089,603.6
3,0,1,-3,16,603.6
4,2,5,-9,196,603.6
5,1,6,-6,144,603.6
6,2,10,-9,361,603.6
7,7,7,-24,961,603.6
8,9,0,-30,900,603.6
9,5,9,-18,729,603.6


In [9]:
"""Note that d[function/voi] below represents partial derivative of J w.r.t 
the function or voi next to d"""

"""dJ/dloss"""
df4 = mean.backward(1)

"""dJ/dy_hat & dJ/dy"""
df3 = square.backward(df4)
dy, df2 = subtract.backward(df3)

"""dJ/dw, dJ/db & dJ/dx"""
df1, db = add.backward(df2)
dw, dx = multiply.backward(df1)

In [10]:
Backward_Propagate = {
    '$\frac{\delta J}{\delta loss$}$': df4, 
    '$\frac{\delta J}{\delta \hat{y}}$': df2, 
    '$\frac{\delta J}{\delta w}$': dw, 
    '$\frac{\delta J}{\delta b}$': db
}

df = pd.DataFrame(data = Backward_Propagate)
df

,$ rac{\delta J}{\delta loss$}$,$ rac{\delta J}{\delta \hat{y}}$,$ rac{\delta J}{\delta w}$,$ rac{\delta J}{\delta b}$
0,0.1,-7.6,-76.0,-7.6
1,0.1,-2.8,-2.8,-2.8
2,0.1,-6.6,-46.2,-6.6
3,0.1,-0.8,-0.0,-0.8
4,0.1,-2.8,-5.6,-2.8
5,0.1,-2.4,-2.4,-2.4
6,0.1,-3.8,-7.6,-3.8
7,0.1,-6.2,-43.4,-6.2
8,0.1,-6.0,-54.0,-6.0
9,0.1,-5.4,-27.0,-5.4


In [13]:
epochs = 5 
lr = 0.01 
 
weight_update = [] 
bias_update = []
loss_update = []

for epoch in range(epochs):
    #shuffle
    """Forward Propagate"""
    f1 = multiply.forward(weight, X)
    f2 = add.forward(f1, bias)
    f3 = subtract.forward(Y, f2)
    f4 = square.forward(f3)
    f5 = mean.forward(f4)
        
    """Backward Propagate"""
    df4 = mean.backward(1)
    df3 = square.backward(df4)
    dy, df2 = subtract.backward(df3)
    df1, db = add.backward(df2)
    dw, dx = multiply.backward(df1)
        
    dw, db = np.sum(dw), np.sum(db)
        
    weight -= lr*dw
    bias -= lr*db
        
    weight_update.append(weight)
    bias_update.append(bias)
    loss_update.append(f4)

**Affine, Loss, Cost**

**Example w/out batch (n_features = n, m_samples = m)**

In [ ]:
for epoch in (epochs): 
    #shuffle columns(samples) of X 
    x, y = X, Y 
    
    y_hat = affine.forward(weight, bias, x)
    loss = loss.forward(y, y_hat)
    cost = cost.forward(loss)
            
    dloss = cost.backward(sample_size = m_samples, 1)
    dy_hat = loss.backward(dloss)
    dw, db = affine.backward(dy_hat)
            
    dw, db = np.sum(dw), np.sum(db)
            
    weight -= lr*dw 
    bias -= lr*db
            
    weight_update.append(weight)
    bias_update.append(bias)
    epoch_cost_update.append(cost)
    

**Example w/ batch (n_features = n, m_samples = m, batch_size = bs)**

In [ ]:
for epoch in (epochs): 
    #shuffle columns(samples) of X  
    batches = batch(X, Y, batch_size = int)
    for batch in (batches):  
        x, y = X_batch_i, Y_batch_i 
            
        y_hat = affine.forward(weight, bias, x)
        loss = loss.forward(y, y_hat)
        cost = cost.forward(loss)
            
        dloss = cost.backward(sample_size = batch_size, 1)
        dy_hat = loss.backward(dloss)
        dw, db = affine.backward(dy_hat)
            
        dw, db = np.sum(dw), np.sum(db)
            
        weight -= lr*dw 
        bias -= lr*db
            
        weight_update.append(weight)
        bias_update.append(bias)
        batch_cost_update.append(cost)
        
    epoch_cost_update.append(cost)

* summarize.weight(feature = int) : returns a graph of how weight of a particular feature is updated at every interation. x-axis should be epoch, and y-axis should be weight 
* summarize.bias() : returns a graph of how bias of the equation is updated at every iteration. x-axis should be epoch, and y-axis should be bias 
* summarize.cost() : returns a graph of how cost is updated every iteration. x-axis should be epoch, and y-axis should be cost 

* summarize.total() : 